# Operationalisation of EVD Index

In [ ]:
# Importing libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as sm

# Load dataset & creating variable name as "EVD"
EVD = pd.read_csv("\\file name")
EVD

In [ ]:
def calculate_environmental_dynamism(firm_data):
    # Extract relevant columns
    years = [str(year) for year in range(1996, 2024)]
    
    # Initialize a dictionary to store results for each firm
    environmental_dynamism = {}
    
    # Iterate over each row (firm) in the dataset
    for index, row in firm_data.iterrows():
        start_year = str(row['Start Year']).strip()  # Strip any extra characters
        firm_name = row['Deal Name']
        
        # Filter industry revenue data for preceding 5 years
        if start_year not in years:
            continue  # Skip if the start year is not in the expected range
        
        start_index = years.index(start_year)
        end_index = start_index - 5
        
        if end_index < 0:
            continue  # Skip if there are less than 5 preceding years
        
        revenue_values = row[years[end_index:start_index]]
        time_values = list(range(end_index + 1996, start_index + 1996))  # Define time values here
        
        # Calculate mean industry revenue for preceding 5 years
        mean_revenue = revenue_values.mean()
        
        print("Mean Revenue Values:", revenue_values)
        print("Time Values:", time_values)

        # Perform linear regression using statsmodels to get the standard error
        X = sm.add_constant(time_values)  # Add a constant term for the intercept
        y = revenue_values.astype(float)  # Ensure y is in float
        
        model = sm.OLS(y, X).fit()
        slope_coefficient = model.params[1]
        slope_standard_error = model.bse[1]
        
        # Calculate environmental dynamism
        if mean_revenue != 0:
            environmental_dynamism[firm_name] = slope_standard_error / mean_revenue
        else:
            environmental_dynamism[firm_name] = np.nan
    
    return environmental_dynamism

In [ ]:
# Convert year columns to numeric, coercing errors to NaN
year_columns = [str(year) for year in range(1996, 2021)]
for col in year_columns:
    EVD[col] = pd.to_numeric(EVD[col], errors='coerce')

# Identify rows where any of these year columns contain NaN values
non_numeric_rows = EVD[year_columns].isna().any(axis=1)

# Display rows with non-numeric values in year columns
if non_numeric_rows.any():
    print("Rows with non-numeric values in year columns:")
    print(EVD[non_numeric_rows])
else:
    print("No non-numeric values found in year columns.")


In [ ]:
# Calculate environmental dynamism for all firms using EVD DataFrame
environmental_dynamism_result = calculate_environmental_dynamism(EVD)

In [ ]:
# Print environmental dynamism for each firm
for firm, dynamism in environmental_dynamism_result.items():
    print(f"Firm: {firm}, Environmental Dynamism: {dynamism}")

In [ ]:
# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(list(environmental_dynamism_result.items()), columns=['Firm', 'Environmental Dynamism'])

# Save the DataFrame to a CSV file
result_df.to_csv('\\outputfile', index=False)